In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
Price = pd.read_csv(r"C:\Users\USER\Documents\Tradingstuff\rawdatabinanceBTCUSDT-1m-data.csv")

Price.tail()

,timestamp,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
1149628,2025-03-09 09:35:00,85858.00,85858.01,85845.92,85846.00,5.16517,1741512959999,4.434211e+05,606,0.40038,34371.082235,0
1149629,2025-03-09 09:36:00,85845.99,85846.00,85796.46,85813.50,22.54549,1741513019999,1.934894e+06,1647,1.44188,123720.170463,0
1149630,2025-03-09 09:37:00,85813.51,85813.51,85783.46,85783.47,9.61634,1741513079999,8.250376e+05,629,0.41522,35627.912764,0
1149631,2025-03-09 09:38:00,85783.47,85792.72,85782.00,85792.72,1.85422,1741513139999,1.590602e+05,785,1.14784,98464.715418,0
1149632,2025-03-09 09:39:00,85792.72,85792.72,85773.95,85773.95,5.46657,1741513199999,4.689517e+05,276,0.02687,2305.173657,0


In [3]:
PI = pd.read_csv(r"C:\Users\USER\Documents\Tradingstuff\btcusd_premium_index_klines_1min.csv", parse_dates=['Open time'])
PI.set_index('Open time', inplace=True)
PI.tail()

,Open,High,Low,Close,Close time
Open time,,,,,
2025-03-08 15:56:00,-0.000565,-0.000375,-0.000653,-0.000473,2025-03-08 15:56:59.999
2025-03-08 15:57:00,-0.000506,-0.000464,-0.000619,-0.000536,2025-03-08 15:57:59.999
2025-03-08 15:58:00,-0.000564,-0.000507,-0.000772,-0.000672,2025-03-08 15:58:59.999
2025-03-08 15:59:00,-0.000593,-0.000437,-0.000593,-0.000534,2025-03-08 15:59:59.999
2025-03-08 16:00:00,-0.000448,-0.000435,-0.000711,-0.000614,2025-03-08 16:00:59.999


In [4]:
columns_to_drop = ['Open', 'Close', 'Close time']
PI.drop(columns=columns_to_drop, axis=1, inplace=True)

# Create boolean masks for the conditions
mask_neg = (PI['High'] < 0) & (PI['Low'] < 0)   # Both negative
mask_pos = (PI['High'] > 0) & (PI['Low'] > 0)   # Both positive

# Compute 'High/Low' vectorized:
# - If both are negative, use the minimum
# - If both are positive, use the maximum
# - Otherwise (mixed signs), use the average
PI['High/Low'] = np.where(
    mask_neg,
    np.minimum(PI['High'], PI['Low']),
    np.where(
        mask_pos,
        np.maximum(PI['High'], PI['Low']),
        (PI['High'] + PI['Low']) / 2
    )
)

# Drop the original 'High' and 'Low' columns
PI.drop(['High', 'Low'], axis=1, inplace=True)

# Resample the DataFrame to hourly data with ohlc aggregation
PI_hr = PI.resample('H').ohlc()

# Display the first few rows
PI_hr.head()


High/Low                              
                         open      high       low     close
Open time                                                  
2022-12-31 16:00:00 -0.000380 -0.000163 -0.000424 -0.000366
2022-12-31 17:00:00 -0.000357 -0.000313 -0.000561 -0.000500
2022-12-31 18:00:00 -0.000503 -0.000323 -0.000572 -0.000395
2022-12-31 19:00:00 -0.000520 -0.000203 -0.000527 -0.000358
2022-12-31 20:00:00 -0.000436 -0.000211 -0.000467 -0.000298

In [5]:
PI_hr.columns = ['_'.join(col).strip() for col in PI_hr.columns.values]

PI_hr.head()

,High/Low_open,High/Low_high,High/Low_low,High/Low_close
Open time,,,,
2022-12-31 16:00:00,-0.000380,-0.000163,-0.000424,-0.000366
2022-12-31 17:00:00,-0.000357,-0.000313,-0.000561,-0.000500
2022-12-31 18:00:00,-0.000503,-0.000323,-0.000572,-0.000395
2022-12-31 19:00:00,-0.000520,-0.000203,-0.000527,-0.000358
2022-12-31 20:00:00,-0.000436,-0.000211,-0.000467,-0.000298


In [6]:
# Drop the existing columns
existing_columns_to_drop = ['High/Low_open', 'High/Low_close']
PI_hr.drop(existing_columns_to_drop, axis=1, inplace=True)

# Create boolean masks for the conditions:
# - If both High/Low_high and High/Low_low are negative, use the minimum.
# - If both are positive, use the maximum.
# - Otherwise, use the average.
mask_neg = (PI_hr['High/Low_high'] < 0) & (PI_hr['High/Low_low'] < 0)
mask_pos = (PI_hr['High/Low_high'] > 0) & (PI_hr['High/Low_low'] > 0)

PI_hr['High/Low'] = np.where(
    mask_neg,
    np.minimum(PI_hr['High/Low_high'], PI_hr['High/Low_low']),
    np.where(
        mask_pos,
        np.maximum(PI_hr['High/Low_high'], PI_hr['High/Low_low']),
        (PI_hr['High/Low_high'] + PI_hr['High/Low_low']) / 2
    )
)

# Drop the original 'High/Low_high' and 'High/Low_low' columns
droppie2 = ['High/Low_high', 'High/Low_low']
PI_hr.drop(droppie2, axis=1, inplace=True)

# Display the first few rows
PI_hr.head()


,High/Low
Open time,
2022-12-31 16:00:00,-0.000424
2022-12-31 17:00:00,-0.000561
2022-12-31 18:00:00,-0.000572
2022-12-31 19:00:00,-0.000527
2022-12-31 20:00:00,-0.000467


In [7]:
IV = pd.read_csv(r"C:\Users\USER\Documents\Tradingstuff\BTC_IV_Deribit_hour.csv")
IV.head()

,timestamp,open,high,low,close
0,2023-05-31 16:00:00,44.98,45.03,44.88,45.02
1,2023-05-31 17:00:00,45.02,45.03,44.82,44.98
2,2023-05-31 18:00:00,44.98,44.98,44.78,44.78
3,2023-05-31 19:00:00,44.78,44.78,44.49,44.57
4,2023-05-31 20:00:00,44.57,44.74,44.56,44.66


In [8]:
IV.drop(['open', 'high', 'low'], axis=1, inplace=True)
IV.head()

,timestamp,close
0,2023-05-31 16:00:00,45.02
1,2023-05-31 17:00:00,44.98
2,2023-05-31 18:00:00,44.78
3,2023-05-31 19:00:00,44.57
4,2023-05-31 20:00:00,44.66


In [9]:
droppieee = ['open','close_time','quote_av','trades','tb_base_av','tb_quote_av','ignore']
Price.drop(droppieee, axis=1, inplace=True)
Price.head()

,timestamp,high,low,close,volume
0,2023-01-01 00:00:00,16544.76,16538.45,16543.67,83.08143
1,2023-01-01 00:01:00,16544.41,16538.48,16539.31,80.45300
2,2023-01-01 00:02:00,16541.17,16534.52,16536.43,62.90197
3,2023-01-01 00:03:00,16537.28,16531.00,16533.65,115.71894
4,2023-01-01 00:04:00,16536.08,16527.51,16535.38,144.45369


In [10]:
Price.set_index('timestamp', inplace=True)
Price.index = pd.to_datetime(Price.index)
Price_hr = Price.resample('H').ohlc()
Price_hr.head()

high                                     low  \
                         open      high       low     close      open   
timestamp                                                               
2023-01-01 00:00:00  16544.76  16545.70  16516.79  16531.10  16538.45   
2023-01-01 01:00:00  16529.93  16556.80  16529.93  16553.43  16525.78   
2023-01-01 02:00:00  16551.53  16559.77  16542.18  16548.60  16542.23   
2023-01-01 03:00:00  16548.19  16548.19  16523.10  16534.56  16531.47   
2023-01-01 04:00:00  16534.23  16535.97  16519.22  16522.81  16531.60   

                                                      close            \
                         high       low     close      open      high   
timestamp                                                               
2023-01-01 00:00:00  16542.74  16508.39  16528.94  16543.67  16544.77   
2023-01-01 01:00:00  16549.57  16525.78  16549.43  16528.04  16553.35   
2023-01-01 02:00:00  16556.00  16538.14  16546.71  16542.44  16558.65   
2023-01-01 03:00:00  16534.70  16518.21  16532.51  16533.94  16537.09   
2023-01-01 04:00:00  16533.07  16511.92  16511.92  16533.12  16533.71   

                                            volume                       \
                          low     close       open       high       low   
timestamp                                                                 
2023-01-01 00:00:00  16514.65  16529.67   83.08143  165.72038  24.30325   
2023-01-01 01:00:00  16528.04  16551.47   84.85522  165.19189  25.86477   
2023-01-01 02:00:00  16538.77  16548.19  155.52693  218.31312  22.48394   
2023-01-01 03:00:00  16520.71  16533.04  344.42893  344.42893  23.74968   
2023-01-01 04:00:00  16517.41  16521.85  100.03658  293.33574  15.30673   

                                
                         close  
timestamp                       
2023-01-01 00:00:00   33.81472  
2023-01-01 01:00:00   37.22213  
2023-01-01 02:00:00   39.09584  
2023-01-01 03:00:00   41.13348  
2023-01-01 04:00:00  173.39683

In [11]:
Price_hr.columns = ['_'.join(col).strip() for col in Price_hr.columns.values]
Price_hr.drop(['close_open', 'close_high', 'close_low','high_open','high_high','high_low','low_open','low_high','low_low',
              'volume_open','volume_high','volume_low'], axis=1, inplace=True)
Price_hr.head()

,high_close,low_close,close_close,volume_close
timestamp,,,,
2023-01-01 00:00:00,16531.10,16528.94,16529.67,33.81472
2023-01-01 01:00:00,16553.43,16549.43,16551.47,37.22213
2023-01-01 02:00:00,16548.60,16546.71,16548.19,39.09584
2023-01-01 03:00:00,16534.56,16532.51,16533.04,41.13348
2023-01-01 04:00:00,16522.81,16511.92,16521.85,173.39683


In [12]:
Price_hr['log_return'] = np.log(Price_hr['close_close'] / Price_hr['close_close'].shift(1))
Price_hr.dropna(axis=0, inplace=True
               )
Price_hr.head()

,high_close,low_close,close_close,volume_close,log_return
timestamp,,,,,
2023-01-01 01:00:00,16553.43,16549.43,16551.47,37.22213,0.001318
2023-01-01 02:00:00,16548.60,16546.71,16548.19,39.09584,-0.000198
2023-01-01 03:00:00,16534.56,16532.51,16533.04,41.13348,-0.000916
2023-01-01 04:00:00,16522.81,16511.92,16521.85,173.39683,-0.000677
2023-01-01 05:00:00,16532.97,16529.29,16530.71,51.34683,0.000536


In [13]:
IV.set_index('timestamp',inplace=True)
IV.index.name = 'Open time'
IV.rename(columns = {'close' : 'smoothed_IV'},inplace=True)
IV.index = pd.to_datetime(IV.index)
IV.head()

,smoothed_IV
Open time,
2023-05-31 16:00:00,45.02
2023-05-31 17:00:00,44.98
2023-05-31 18:00:00,44.78
2023-05-31 19:00:00,44.57
2023-05-31 20:00:00,44.66


In [14]:
PI_hr.rename(columns = {'High/Low' : 'min_max_PI'},inplace=True)
PI_hr.head()

,min_max_PI
Open time,
2022-12-31 16:00:00,-0.000424
2022-12-31 17:00:00,-0.000561
2022-12-31 18:00:00,-0.000572
2022-12-31 19:00:00,-0.000527
2022-12-31 20:00:00,-0.000467


In [15]:
Price_hr.rename(columns = {'close_close' : 'close_price','high_close' : 'high',	'low_close' : 'low', 'volume_close' : 'volume'	},inplace=True)
Price_hr.head()

,high,low,close_price,volume,log_return
timestamp,,,,,
2023-01-01 01:00:00,16553.43,16549.43,16551.47,37.22213,0.001318
2023-01-01 02:00:00,16548.60,16546.71,16548.19,39.09584,-0.000198
2023-01-01 03:00:00,16534.56,16532.51,16533.04,41.13348,-0.000916
2023-01-01 04:00:00,16522.81,16511.92,16521.85,173.39683,-0.000677
2023-01-01 05:00:00,16532.97,16529.29,16530.71,51.34683,0.000536


In [16]:
df = pd.concat([Price_hr, IV, PI_hr], axis=1 ,join='inner') #only join overlapping date
df.head()

,high,low,close_price,volume,log_return,smoothed_IV,min_max_PI
2023-05-31 16:00:00,26936.27,26936.00,26936.01,4.71903,0.000909,45.02,-0.000764
2023-05-31 17:00:00,27055.40,27048.14,27055.40,12.97853,0.004423,44.98,-0.000976
2023-05-31 18:00:00,27041.45,27037.05,27037.06,4.38032,-0.000678,44.78,-0.000702
2023-05-31 19:00:00,27068.90,27000.20,27010.57,53.44380,-0.000980,44.57,-0.000788
2023-05-31 20:00:00,27114.20,27114.19,27114.20,7.44980,0.003829,44.66,-0.000785


In [17]:
df['rolling_std_30'] = df['log_return'].rolling(window=720).std()
df['rolling_annual_vol_30'] = 100 * df['rolling_std_30'] * np.sqrt(8760)
df['Vol_Spread'] = df['smoothed_IV'] - df['rolling_annual_vol_30']
df

,high,low,close_price,volume,log_return,smoothed_IV,min_max_PI,rolling_std_30,rolling_annual_vol_30,Vol_Spread
2023-05-31 16:00:00,26936.27,26936.00,26936.01,4.71903,0.000909,45.02,-0.000764,NaN,NaN,NaN
2023-05-31 17:00:00,27055.40,27048.14,27055.40,12.97853,0.004423,44.98,-0.000976,NaN,NaN,NaN
2023-05-31 18:00:00,27041.45,27037.05,27037.06,4.38032,-0.000678,44.78,-0.000702,NaN,NaN,NaN
2023-05-31 19:00:00,27068.90,27000.20,27010.57,53.44380,-0.000980,44.57,-0.000788,NaN,NaN,NaN
2023-05-31 20:00:00,27114.20,27114.19,27114.20,7.44980,0.003829,44.66,-0.000785,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2025-03-08 12:00:00,86241.84,86208.00,86209.72,2.47277,0.002949,52.37,-0.000772,0.006718,62.877272,-10.507272
2025-03-08 13:00:00,86331.12,86318.42,86327.96,4.96618,0.001371,52.70,-0.000897,0.006717,62.871974,-10.171974
2025-03-08 14:00:00,86361.69,86324.00,86361.69,3.18146,0.000391,52.77,-0.001057,0.006711,62.815626,-10.045626
2025-03-08 15:00:00,86400.00,86382.65,86400.00,2.00251,0.000444,52.83,-0.000878,0.006709,62.791896,-9.961896


In [18]:
window_size = 720
df['rolling_mean_PI'] = df['min_max_PI'].rolling(window=window_size).mean()
df['rolling_std_PI'] = df['min_max_PI'].rolling(window=window_size).std()
df['z_score_PI'] = (df['min_max_PI'] - df['rolling_mean_PI']) / df['rolling_std_PI']

df['rol_mean_Vol_Spread'] = df['Vol_Spread'].rolling(window=window_size).mean()
df['rol_std_Vol_Spread'] = df['Vol_Spread'].rolling(window=window_size).std()
df['z_score_Vol_Spread'] = (df['Vol_Spread'] - df['rol_mean_Vol_Spread']) / df['rol_std_Vol_Spread']

df['PI_VolSpread_zdiff'] = df['z_score_PI'] + df['z_score_Vol_Spread']

df

,high,low,close_price,volume,log_return,smoothed_IV,min_max_PI,rolling_std_30,rolling_annual_vol_30,Vol_Spread,rolling_mean_PI,rolling_std_PI,z_score_PI,rol_mean_Vol_Spread,rol_std_Vol_Spread,z_score_Vol_Spread,PI_VolSpread_zdiff
2023-05-31 16:00:00,26936.27,26936.00,26936.01,4.71903,0.000909,45.02,-0.000764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-31 17:00:00,27055.40,27048.14,27055.40,12.97853,0.004423,44.98,-0.000976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-31 18:00:00,27041.45,27037.05,27037.06,4.38032,-0.000678,44.78,-0.000702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-31 19:00:00,27068.90,27000.20,27010.57,53.44380,-0.000980,44.57,-0.000788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-31 20:00:00,27114.20,27114.19,27114.20,7.44980,0.003829,44.66,-0.000785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-08 12:00:00,86241.84,86208.00,86209.72,2.47277,0.002949,52.37,-0.000772,0.006718,62.877272,-10.507272,-0.000929,0.000328,0.478702,-1.356775,4.451264,-2.055707,-1.577005
2025-03-08 13:00:00,86331.12,86318.42,86327.96,4.96618,0.001371,52.70,-0.000897,0.006717,62.871974,-10.171974,-0.000929,0.000328,0.096880,-1.363140,4.460591,-1.974813,-1.877933
2025-03-08 14:00:00,86361.69,86324.00,86361.69,3.18146,0.000391,52.77,-0.001057,0.006711,62.815626,-10.045626,-0.000928,0.000327,-0.394282,-1.369053,4.469279,-1.941381,-2.335662
2025-03-08 15:00:00,86400.00,86382.65,86400.00,2.00251,0.000444,52.83,-0.000878,0.006709,62.791896,-9.961896,-0.000928,0.000327,0.153394,-1.374614,4.477480,-1.917883,-1.764488


In [19]:
horizons = [4,6,8,10,12,14,16]  # for instance

threshold_return = 0.02  # 2%

for h in horizons:
    # Compute forward log return: ln(P[t+h] / P[t])
    df[f'fwd_ret_{h}'] = np.log(df['close_price'].shift(-h) / df['close_price'])
    
    # Create a ternary label:
    # 1 if forward log return > 2%, -1 if forward log return < -2%, otherwise 0.
    conditions = [
        df[f'fwd_ret_{h}'] > threshold_return,
        df[f'fwd_ret_{h}'] < -threshold_return
    ]
    choices = [1, -1]
    df[f'label_{h}'] = np.select(conditions, choices, default=0)

# Because the last h rows are NaN for fwd_ret, drop rows with NaN in any fwd_ret column
df.dropna(subset=[f'fwd_ret_{h}' for h in horizons], how='all', inplace=True)

# Print statistics for each horizon
for h in horizons:
    ret_col = f'fwd_ret_{h}'
    label_col = f'label_{h}'
    
    # Filter out rows where this horizon is NaN
    valid_rows = df[ret_col].dropna()
    
    if valid_rows.empty:
        print(f"No valid data for horizon {h}, skipping...\n")
        continue
    
    print(f"=== Horizon {h} ===")
    print(f"Forward Log Return '{ret_col}' Stats:\n", valid_rows.describe())
    
    # Label distribution
    label_mean = df.loc[valid_rows.index, label_col].mean()
    label_counts = df.loc[valid_rows.index, label_col].value_counts().sort_index()
    
    print(f"\nLabel '{label_col}' Distribution:")
    print(f" - Proportion of 1: {(df.loc[valid_rows.index, label_col] == 1).mean():.2%}")
    print(f" - Proportion of -1: {(df.loc[valid_rows.index, label_col] == -1).mean():.2%}")
    print(f" - Proportion of 0: {(df.loc[valid_rows.index, label_col] == 0).mean():.2%}")
    print(f" - Count of labels:\n{label_counts}")
    
    print("\n" + "-"*40 + "\n")

=== Horizon 4 ===
Forward Log Return 'fwd_ret_4' Stats:
 count    15525.000000
mean         0.000299
std          0.010270
min         -0.086963
25%         -0.003733
50%          0.000199
75%          0.004317
max          0.093313
Name: fwd_ret_4, dtype: float64

Label 'label_4' Distribution:
 - Proportion of 1: 3.32%
 - Proportion of -1: 2.93%
 - Proportion of 0: 93.75%
 - Count of labels:
label_4
-1      455
 0    14555
 1      515
Name: count, dtype: int64

----------------------------------------

=== Horizon 6 ===
Forward Log Return 'fwd_ret_6' Stats:
 count    15523.000000
mean         0.000448
std          0.012539
min         -0.104602
25%         -0.004583
50%          0.000284
75%          0.005468
max          0.102885
Name: fwd_ret_6, dtype: float64

Label 'label_6' Distribution:
 - Proportion of 1: 5.12%
 - Proportion of -1: 4.55%
 - Proportion of 0: 90.32%
 - Count of labels:
label_6
-1      707
 0    14021
 1      795
Name: count, dtype: int64

------------------------

In [20]:
# Remove the forward return columns after using them
for h in horizons:
    df.drop(columns=[f'fwd_ret_{h}'], inplace=True)

# (Optional) Display the remaining DataFrame columns to verify removal
print("Remaining columns in DataFrame:", df.columns.tolist())


Remaining columns in DataFrame: ['high', 'low', 'close_price', 'volume', 'log_return', 'smoothed_IV', 'min_max_PI', 'rolling_std_30', 'rolling_annual_vol_30', 'Vol_Spread', 'rolling_mean_PI', 'rolling_std_PI', 'z_score_PI', 'rol_mean_Vol_Spread', 'rol_std_Vol_Spread', 'z_score_Vol_Spread', 'PI_VolSpread_zdiff', 'label_4', 'label_6', 'label_8', 'label_10', 'label_12', 'label_14', 'label_16']


In [21]:
columns_to_lag = ['smoothed_IV', 'min_max_PI', 'Vol_Spread']

# Define the maximum lag
max_lag = 6

# Create lagged columns for each specified column
for col in columns_to_lag:
    for lag in range(1, max_lag + 1):
        df[f"{col}_lag_{lag}"] = df[col].shift(lag)
df.head()

,high,low,close_price,volume,log_return,smoothed_IV,min_max_PI,rolling_std_30,rolling_annual_vol_30,Vol_Spread,...,min_max_PI_lag_3,min_max_PI_lag_4,min_max_PI_lag_5,min_max_PI_lag_6,Vol_Spread_lag_1,Vol_Spread_lag_2,Vol_Spread_lag_3,Vol_Spread_lag_4,Vol_Spread_lag_5,Vol_Spread_lag_6
2023-05-31 16:00:00,26936.27,26936.00,26936.01,4.71903,0.000909,45.02,-0.000764,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-31 17:00:00,27055.40,27048.14,27055.40,12.97853,0.004423,44.98,-0.000976,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-31 18:00:00,27041.45,27037.05,27037.06,4.38032,-0.000678,44.78,-0.000702,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-31 19:00:00,27068.90,27000.20,27010.57,53.44380,-0.000980,44.57,-0.000788,NaN,NaN,NaN,...,-0.000764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-31 20:00:00,27114.20,27114.19,27114.20,7.44980,0.003829,44.66,-0.000785,NaN,NaN,NaN,...,-0.000976,-0.000764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
def compute_macd(df, price_col='close_price', fast_period=12, slow_period=26, signal_period=9):
    
    # Compute the fast and slow EMAs
    ema_fast = df[price_col].ewm(span=fast_period, adjust=False).mean()
    ema_slow = df[price_col].ewm(span=slow_period, adjust=False).mean()
    
    # MACD line
    macd_line = ema_fast - ema_slow
    
    # Signal line (EMA of MACD)
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    
    # MACD histogram (difference between MACD line and signal line)
    macd_hist = macd_line - signal_line
    
    return macd_line, signal_line, macd_hist

def add_macd_to_df(df, price_col='close_price', fast_period=12, slow_period=26, signal_period=9):
   
    macd_line, signal_line, macd_hist = compute_macd(
        df, 
        price_col=price_col,
        fast_period=fast_period,
        slow_period=slow_period,
        signal_period=signal_period
    )
    df['MACD_Line'] = macd_line
    df['MACD_Signal'] = signal_line
    df['MACD_Histogram'] = macd_hist
    return df
    
def compute_roc(df, price_col='close_price', period=10):
    """
    Computes the Rate of Change (ROC).
    ROC = (Current Price - Price n periods ago) / Price n periods ago
    """
    return df[price_col].pct_change(period)

def compute_williams_r(df, high_col='high', low_col='low', close_col='close_price', period=14):
    """
    Computes the Williams %R indicator.
    %R = (Highest High - Close) / (Highest High - Lowest Low) * -100
    """
    highest_high = df[high_col].rolling(window=period).max()
    lowest_low = df[low_col].rolling(window=period).min()
    wr = -100 * (highest_high - df[close_col]) / (highest_high - lowest_low)
    return wr

def compute_mfi(df, high_col='high', low_col='low', close_col='close_price', volume_col='volume', period=14):
    """
    Computes the Money Flow Index (MFI).
    Typical Price = (High + Low + Close) / 3
    Money Flow = Typical Price * Volume
    """
    typical_price = (df[high_col] + df[low_col] + df[close_col]) / 3
    money_flow = typical_price * df[volume_col]
    
    # Determine positive and negative money flows
    # Positive if current typical price > previous typical price, negative if less.
    tp_diff = typical_price.diff()
    pos_mf = money_flow.where(tp_diff > 0, 0)
    neg_mf = money_flow.where(tp_diff < 0, 0)
    
    # Rolling sum over the specified period
    pos_mf_sum = pos_mf.rolling(window=period).sum()
    neg_mf_sum = neg_mf.rolling(window=period).sum().abs()
    
    mfi = 100 - (100 / (1 + pos_mf_sum / neg_mf_sum))
    return mfi

def compute_obv(df, close_col='close_price', volume_col='volume'):
    """
    Computes the On-Balance Volume (OBV).
    """
    obv = [0]
    for i in range(1, len(df)):
        if df[close_col].iloc[i] > df[close_col].iloc[i-1]:
            obv.append(obv[-1] + df[volume_col].iloc[i])
        elif df[close_col].iloc[i] < df[close_col].iloc[i-1]:
            obv.append(obv[-1] - df[volume_col].iloc[i])
        else:
            obv.append(obv[-1])
    return pd.Series(obv, index=df.index)

def compute_force_index(df, price_col='close_price', volume_col='volume', period=1):
    """
    Computes the Force Index.
    Force Index = Volume * (Current Price - Previous Price)
    """
    force = df[price_col].diff(period) * df[volume_col]
    return force

def compute_volume_roc(df, volume_col='volume', period=10):
    """
    Computes the Volume Rate of Change (Volume ROC).
    Volume ROC = (Current Volume - Volume n periods ago) / Volume n periods ago
    """
    return df[volume_col].pct_change(period)

def compute_rsi(series, period):
    """
    Compute the Relative Strength Index (RSI) for a given price series.
    This implementation uses a simple moving average for the gains and losses.
    
    Parameters:
        series (pd.Series): Price series (e.g., close prices).
        period (int): Lookback period for the RSI.
    
    Returns:
        pd.Series: RSI values.
    """
    delta = series.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    # Calculate the rolling average of gains and losses.
    avg_gain = gain.rolling(window=period, min_periods=period).mean()
    avg_loss = loss.rolling(window=period, min_periods=period).mean()
    
    # Avoid division by zero
    rs = avg_gain / (avg_loss + 1e-10)
    rsi = 100 - (100 / (1 + rs))
    return rsi
# Compute and add these features to the DataFrame
df['ROC'] = compute_roc(df, price_col='close_price', period=10)
df['Williams_%R'] = compute_williams_r(df, period=14)
df['MFI'] = compute_mfi(df, period=14)
df['OBV'] = compute_obv(df, close_col='close_price', volume_col='volume')
df['Force_Index'] = compute_force_index(df, price_col='close_price', volume_col='volume')
df['Volume_ROC'] = compute_volume_roc(df, volume_col='volume', period=10)
# Define short and long RSI periods (similar to PineScript inputs)
shortRSI_period = 7
longRSI_period = 35

# Calculate the short and long RSIs
df['short_RSI'] = compute_rsi(df['close_price'], period=shortRSI_period)
df['long_RSI'] = compute_rsi(df['close_price'], period=longRSI_period)

# Calculate the difference between the long and short RSIs
df['RSI_diff'] = df['long_RSI'] - df['short_RSI']
# Add MACD columns
df = add_macd_to_df(df)
df

,high,low,close_price,volume,log_return,smoothed_IV,min_max_PI,rolling_std_30,rolling_annual_vol_30,Vol_Spread,...,MFI,OBV,Force_Index,Volume_ROC,short_RSI,long_RSI,RSI_diff,MACD_Line,MACD_Signal,MACD_Histogram
2023-05-31 16:00:00,26936.27,26936.00,26936.01,4.71903,0.000909,45.02,-0.000764,NaN,NaN,NaN,...,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
2023-05-31 17:00:00,27055.40,27048.14,27055.40,12.97853,0.004423,44.98,-0.000976,NaN,NaN,NaN,...,NaN,12.978530,1549.506697,NaN,NaN,NaN,NaN,9.523989,1.904798,7.619191
2023-05-31 18:00:00,27041.45,27037.05,27037.06,4.38032,-0.000678,44.78,-0.000702,NaN,NaN,NaN,...,NaN,8.598210,-80.335069,NaN,NaN,NaN,NaN,15.414248,4.606688,10.807560
2023-05-31 19:00:00,27068.90,27000.20,27010.57,53.44380,-0.000980,44.57,-0.000788,NaN,NaN,NaN,...,NaN,-44.845590,-1415.726262,NaN,NaN,NaN,NaN,17.740300,7.233410,10.506889
2023-05-31 20:00:00,27114.20,27114.19,27114.20,7.44980,0.003829,44.66,-0.000785,NaN,NaN,NaN,...,NaN,-37.395790,772.022774,NaN,NaN,NaN,NaN,27.627317,11.312191,16.315125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-08 08:00:00,86112.50,86070.77,86071.57,10.03545,-0.001458,52.12,-0.000815,0.006723,62.928420,-10.808420,...,17.924705,-293.540022,-1260.151456,-0.571289,48.927993,41.538452,-7.389541,-665.638390,-645.015626,-20.622764
2025-03-08 09:00:00,85943.29,85889.54,85943.28,5.34728,-0.001492,52.04,-0.000826,0.006721,62.904620,-10.864620,...,18.234955,-298.887302,-686.002551,0.947369,42.861964,40.333841,-2.528123,-663.997329,-648.811967,-15.185362
2025-03-08 10:00:00,86140.51,86121.91,86123.19,2.67768,0.002091,51.89,-0.000839,0.006720,62.892348,-11.002348,...,30.716063,-296.209622,481.741409,-0.837429,56.464744,41.500236,-14.964508,-640.792881,-647.208150,6.415268
2025-03-08 11:00:00,85955.89,85944.76,85955.88,5.95641,-0.001945,52.39,-0.000840,0.006720,62.893982,-10.503982,...,38.742804,-302.166032,-996.566957,0.560205,29.977135,51.174118,21.196983,-628.656942,-643.497908,14.840966


In [23]:
def compute_vol_spread_roc(df, col='Vol_Spread', period=10):
    """
    Computes the Rate of Change (ROC) for the Volatility Spread.
    ROC = (Current Value - Value n periods ago) / Value n periods ago
    """
    return df[col].pct_change(period)

def compute_vol_spread_rsi(df, col='Vol_Spread', period=14):
    """
    Computes the Relative Strength Index (RSI) for the Volatility Spread.
    """
    delta = df[col].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=period, min_periods=period).mean()
    avg_loss = loss.rolling(window=period, min_periods=period).mean()
    
    # Avoid division by zero
    rs = avg_gain / (avg_loss + 1e-10)
    rsi = 100 - (100 / (1 + rs))
    return rsi

def compute_vol_spread_macd(df, col='Vol_Spread', fast_period=12, slow_period=26, signal_period=9):
    """
    Computes the MACD for the Volatility Spread.
    
    Returns:
        macd_line: Fast EMA minus slow EMA of the Volatility Spread
        signal_line: EMA of the macd_line
        macd_hist: Difference between macd_line and signal_line
    """
    ema_fast = df[col].ewm(span=fast_period, adjust=False).mean()
    ema_slow = df[col].ewm(span=slow_period, adjust=False).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    macd_hist = macd_line - signal_line
    return macd_line, signal_line, macd_hist
    
# Compute momentum features for Vol_Spread
df['Vol_Spread_ROC'] = compute_vol_spread_roc(df, col='Vol_Spread', period=10)
df['Vol_Spread_RSI'] = compute_vol_spread_rsi(df, col='Vol_Spread', period=14)
macd_line, signal_line, macd_hist = compute_vol_spread_macd(df, col='Vol_Spread', fast_period=12, slow_period=26, signal_period=9)
df['Vol_Spread_MACD'] = macd_line
df['Vol_Spread_Signal'] = signal_line
df['Vol_Spread_MACD_Hist'] = macd_hist


In [24]:
df.describe()

,high,low,close_price,volume,log_return,smoothed_IV,min_max_PI,rolling_std_30,rolling_annual_vol_30,Vol_Spread,...,long_RSI,RSI_diff,MACD_Line,MACD_Signal,MACD_Histogram,Vol_Spread_ROC,Vol_Spread_RSI,Vol_Spread_MACD,Vol_Spread_Signal,Vol_Spread_MACD_Hist
count,15525.000000,15525.000000,15525.000000,15525.000000,15525.000000,15525.000000,15525.000000,14806.000000,14806.000000,14806.000000,...,15491.000000,15491.000000,15525.000000,15525.000000,15525.000000,14796.000000,14792.000000,14806.000000,14806.000000,14806.000000
mean,57696.272877,57662.578496,57679.518839,22.073196,0.000075,53.519530,-0.000262,0.005024,47.019909,6.887827,...,51.132674,-0.184123,27.248234,27.411363,-0.163129,-0.078573,50.022396,-0.005317,-0.004915,-0.000402
std,23628.997929,23610.492554,23619.338377,36.810537,0.005217,9.740615,0.000995,0.001121,10.494830,8.381794,...,10.828725,20.259000,372.516469,348.546116,117.207109,8.991946,20.182351,0.581769,0.546454,0.177199
min,24929.290000,24860.000000,24923.510000,0.210930,-0.052879,31.470000,-0.006209,0.002361,22.095165,-18.547176,...,11.101919,-65.022689,-2319.577890,-2036.360417,-947.873491,-916.087024,0.000000,-2.665035,-2.446059,-1.355398
25%,35622.000000,35608.000000,35618.540000,5.400700,-0.001945,47.340000,-0.000877,0.004227,39.563113,2.594440,...,43.949337,-14.944757,-106.004489,-103.433684,-45.319341,-0.106002,35.388652,-0.338502,-0.320037,-0.092235
50%,60442.630000,60409.140000,60427.840000,11.375540,0.000077,54.260000,-0.000675,0.004950,46.326839,7.081602,...,51.008095,0.002944,12.710420,13.943849,-1.458127,-0.000442,50.097245,0.009111,0.013559,0.003964
75%,68813.980000,68775.000000,68792.710000,23.777600,0.002179,58.730000,0.000405,0.005825,54.519145,12.696616,...,58.162379,14.306616,177.265130,170.324451,40.589542,0.104576,64.776094,0.319787,0.298270,0.097052
max,108330.030000,108242.280000,108320.010000,1042.574100,0.053802,84.730000,0.014777,0.007496,70.154904,32.440441,...,84.893131,66.287521,2390.735922,1991.815458,884.889483,138.257437,100.000000,2.872007,2.403612,1.312404


In [25]:
df = df.dropna()

In [26]:
df.describe()

,high,low,close_price,volume,log_return,smoothed_IV,min_max_PI,rolling_std_30,rolling_annual_vol_30,Vol_Spread,...,long_RSI,RSI_diff,MACD_Line,MACD_Signal,MACD_Histogram,Vol_Spread_ROC,Vol_Spread_RSI,Vol_Spread_MACD,Vol_Spread_Signal,Vol_Spread_MACD_Hist
count,14087.000000,14087.000000,14087.000000,14087.000000,14087.000000,14087.000000,14087.000000,14087.000000,14087.000000,14087.000000,...,14087.000000,14087.000000,14087.000000,14087.000000,14087.000000,14087.000000,14087.000000,14087.000000,14087.000000,14087.000000
mean,60636.791278,60600.650672,60618.810628,21.788815,0.000077,54.503008,-0.000208,0.005080,47.546478,6.956529,...,51.198205,-0.176269,28.848145,29.027001,-0.178856,-0.084105,50.041906,-0.006401,-0.005903,-0.000499
std,22838.777111,22821.076534,22829.468991,35.092916,0.005340,9.603692,0.001018,0.001117,10.451033,8.583246,...,10.849802,20.197441,389.037781,363.972946,122.474045,9.215286,20.250808,0.591886,0.556372,0.179282
min,25020.670000,24995.930000,25020.660000,0.210930,-0.052879,31.470000,-0.005098,0.002361,22.095165,-18.547176,...,11.101919,-65.022689,-2319.577890,-2036.360417,-947.873491,-916.087024,0.000000,-2.665035,-2.446059,-1.355398
25%,42314.595000,42296.300000,42302.540000,5.399300,-0.002031,49.810000,-0.000878,0.004288,40.129531,2.356934,...,43.840382,-14.829037,-118.912524,-117.805746,-50.275345,-0.104994,35.389635,-0.349412,-0.330923,-0.093887
50%,62103.010000,62073.810000,62090.010000,11.316980,0.000088,55.040000,-0.000657,0.005065,47.407245,7.646512,...,51.044075,-0.043645,15.962604,17.404014,-1.889950,-0.000030,50.155562,0.010093,0.012785,0.004359
75%,69614.315000,69577.300000,69591.790000,23.683120,0.002267,59.250000,0.000659,0.005863,54.878167,12.939817,...,58.410943,14.298464,194.563854,188.314699,46.693619,0.103255,64.766649,0.323912,0.304459,0.097675
max,108330.030000,108242.280000,108320.010000,1042.574100,0.053802,84.730000,0.014777,0.007496,70.154904,32.440441,...,84.476621,66.287521,2390.735922,1991.815458,884.889483,138.257437,100.000000,2.872007,2.403612,1.312404


In [27]:
df.to_csv(r"C:\Users\USER\Documents\Tradingstuff\IV_PI_Stratdf_for_training.csv")

In [28]:
df.tail()

,high,low,close_price,volume,log_return,smoothed_IV,min_max_PI,rolling_std_30,rolling_annual_vol_30,Vol_Spread,...,long_RSI,RSI_diff,MACD_Line,MACD_Signal,MACD_Histogram,Vol_Spread_ROC,Vol_Spread_RSI,Vol_Spread_MACD,Vol_Spread_Signal,Vol_Spread_MACD_Hist
2025-03-08 08:00:00,86112.50,86070.77,86071.57,10.03545,-0.001458,52.12,-0.000815,0.006723,62.928420,-10.808420,...,41.538452,-7.389541,-665.638390,-645.015626,-20.622764,0.286658,0.000000,-1.549088,-1.453640,-0.095447
2025-03-08 09:00:00,85943.29,85889.54,85943.28,5.34728,-0.001492,52.04,-0.000826,0.006721,62.904620,-10.864620,...,40.333841,-2.528123,-663.997329,-648.811967,-15.185362,0.268106,0.000000,-1.561960,-1.475304,-0.086656
2025-03-08 10:00:00,86140.51,86121.91,86123.19,2.67768,0.002091,51.89,-0.000839,0.006720,62.892348,-11.002348,...,41.500236,-14.964508,-640.792881,-647.208150,6.415268,0.238633,0.000000,-1.565233,-1.493290,-0.071943
2025-03-08 11:00:00,85955.89,85944.76,85955.88,5.95641,-0.001945,52.39,-0.000840,0.006720,62.893982,-10.503982,...,51.174118,21.196983,-628.656942,-643.497908,14.840966,0.180496,14.176561,-1.510203,-1.496673,-0.013531
2025-03-08 12:00:00,86241.84,86208.00,86209.72,2.47277,0.002949,52.37,-0.000772,0.006718,62.877272,-10.507272,...,43.453427,-3.895006,-591.735202,-633.145367,41.410165,0.157957,16.057566,-1.450141,-1.487366,0.037225


In [29]:
df.head()

,high,low,close_price,volume,log_return,smoothed_IV,min_max_PI,rolling_std_30,rolling_annual_vol_30,Vol_Spread,...,long_RSI,RSI_diff,MACD_Line,MACD_Signal,MACD_Histogram,Vol_Spread_ROC,Vol_Spread_RSI,Vol_Spread_MACD,Vol_Spread_Signal,Vol_Spread_MACD_Hist
2023-07-30 14:00:00,29391.82,29381.13,29391.81,33.27224,0.003541,35.33,-0.000273,0.002939,27.505899,7.824101,...,51.711811,-22.383681,1.951589,-2.217337,4.168926,0.270454,78.532975,0.479000,0.310619,0.168381
2023-07-30 15:00:00,29349.37,29349.36,29349.37,5.54135,-0.001445,35.24,-0.000876,0.002939,27.508929,7.731071,...,47.436856,-14.262119,4.338682,-0.906133,5.244815,0.248834,75.729101,0.546664,0.357828,0.188836
2023-07-30 16:00:00,29385.67,29375.99,29380.02,39.36677,0.001044,35.76,-0.000683,0.002910,27.239903,8.520097,...,50.755406,-18.931974,8.604481,0.995990,7.608492,0.422478,79.527923,0.656389,0.417540,0.238849
2023-07-30 17:00:00,29396.52,29396.51,29396.52,1.47595,0.000561,35.82,-0.000651,0.002910,27.232957,8.587043,...,54.086696,-17.382354,13.164815,3.429755,9.735060,0.409577,80.409140,0.740217,0.482076,0.258141
2023-07-30 18:00:00,29356.00,29354.99,29355.99,5.72125,-0.001380,35.79,-0.000752,0.002908,27.215382,8.574618,...,54.941333,-2.553402,13.354536,5.414711,7.939825,0.419616,83.735404,0.796467,0.544954,0.251513
